# GeoNergyAna Colab Launcher (Refined Gas‑Phase + ML)

This notebook runs the **enegeo_refined4_gas_ml.py** pipeline end‑to‑end *in‑memory* without saving the script file, protecting your code:
1. Install dependencies (including ML extras)
2. Upload inputs (`.sph`, receptor PDB, ZIP of poses)
3. Unzip and organize inputs
4. Fetch and execute the refined pipeline script from GitHub
5. Inspect results and SHAP/ML outputs
6. Download outputs


In [ ]:
# 1️⃣ Install dependencies
!pip install numpy pandas matplotlib scipy mdtraj biopython openmm scikit-learn lightgbm shap joblib git+https://github.com/openmm/pdbfixer.git

In [ ]:
# 2️⃣ Upload user inputs
from google.colab import files
print("➜ Upload your surface-definition .sph file:")
sph_up = files.upload()
print("➜ Upload your receptor PDB file:")
rec_up = files.upload()
print("➜ Upload a ZIP archive of your poses directory (contains .pdbs):")
poses_up = files.upload()

In [ ]:
# 3️⃣ Unzip poses and organize inputs
import os, zipfile
sph_fname = next(iter(sph_up))
rec_fname = next(iter(rec_up))
poses_zip = next(iter(poses_up))
os.makedirs('user_inputs/poses', exist_ok=True)
os.replace(sph_fname, f'user_inputs/{sph_fname}')
os.replace(rec_fname, f'user_inputs/{rec_fname}')
with zipfile.ZipFile(poses_zip, 'r') as z:
    z.extractall('user_inputs/poses')

In [ ]:
# 4️⃣ Fetch and execute the refined pipeline script
import requests, sys
# Prepare sys.argv for pipeline
args = [
  'enegeo_refined4_gas_ml.py',
  '--sph', f'user_inputs/{sph_fname}',
  '--rec', f'user_inputs/{rec_fname}',
  '--ligdir', 'user_inputs/poses/ligs',  # adjust if your ZIP structure differs
  '--outdir', 'results',
  '--frac_thresh', '0.5',
  '--depth_thresh', '0.3',
  '--vicinity_radius', '2.0',
  '--keep_h',
  '--nsteps', '2000',
  '--dt', '0.001',
  '--interval', '100',
  '--ml',        # enable regression
  '--hpo',       # enable hyperparam search
  '--classify',  # enable classification
  '--verbose'
]
sys.argv = args
# Pull script text
script = requests.get('https://raw.githubusercontent.com/sciencemaths-collab/geonergyana/main/geonergy.py').text
# Execute in‑memory
exec(script)

In [ ]:
# 5️⃣ Inspect outputs
import pandas as pd
from IPython.display import display, Image
print('Generated files in results/:')
!ls results
df = pd.read_csv('results/kept_ligs.csv')
display(df)
# Show key plots
display(Image('results/A_frac_depth.png'))
display(Image('results/C_score_vs_wscore.png'))
display(Image('results/E_score_vs_dG.png'))

In [ ]:
# 6️⃣ Zip & download results
from google.colab import files
!zip -r results.zip results
files.download('results.zip')